In [9]:
import os
import openai
import re
import httpx
from dotenv import load_dotenv
_ = load_dotenv()
from openai import OpenAI


In [10]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [11]:
tool = TavilySearchResults(max_results=2)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [13]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]  ## This will ensure that the messages are added not overwritten

In [14]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [15]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [17]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_wRSnQHrJYaVqTXqX3zb98z6t', 'type': 'tool_call'}
Back to the model!


In [18]:
result

{'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wRSnQHrJYaVqTXqX3zb98z6t', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 153, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BGTtAwoxfDpG5bo4dPQf6muHR4C8h', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b6bc8c55-9af9-4bc7-bdc6-43fd1da472fd-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_wRSnQHrJYaVqTXqX3

In [19]:
result['messages'][-1].content

'The weather in San Francisco for the end of March 2025 is as follows:\n- March 28: 12°C (54°F) with 2.4 mm of precipitation\n- March 29: 13°C (55°F) with 1.8 mm of precipitation\n- March 30: 13°C (55°F) with 0.6 mm of precipitation\n- March 31: 12°C (54°F) with 1.9 mm of precipitation\n\nYou can find more detailed information on the [Climate Data website](https://en.climate-data.org/north-america/united-states-of-america/california/san-francisco-385/t/march-3/).'

In [20]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_4nwmRTJZIs8cwqSxdysr2w14', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_H3UGC6435Pcn6KbrLOnhQwkZ', 'type': 'tool_call'}
Back to the model!


In [21]:
result['messages'][-1].content

'The weather in San Francisco on March 29, 2025, is expected to have a daytime temperature of 13°C (55°F) and a nighttime temperature of 17°C (63°F) with a slight chance of precipitation. \n\nIn Los Angeles on March 29, 2025, the daytime temperature is forecasted to be around 68°F, dropping to 52°F at night. There is a low chance of precipitation with a wind speed of 7 mph and a UV index of 7.'

In [22]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': 'call_GhKQtJDKJkp9hhFK0JjjAbnS', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'where is Kansas City Chiefs headquarters located'}, 'id': 'call_64NPH71G3w0Cx7aw6r6NXV9j', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Missouri 2024'}, 'id': 'call_MUdBtqKnunI5efOmeJHDVix4', 'type': 'tool_call'}
Back to the model!


In [23]:
print(result['messages'][-1].content)

1. The Kansas City Chiefs won the Super Bowl in 2024.
2. The headquarters of the Kansas City Chiefs is located in Kansas City, Missouri.
3. The GDP of Missouri in late 2024 was approximately $455 billion.
